# Quantum Volume

Here we show off using the expectation value functionality of the M3 distribution classes using Quantum Volume (QV) as an example.  Here we formulate QV as an expectation value of a projector onto the heavy-output elements on a distribution.

In [1]:
import numpy as np
from qiskit import *
from qiskit.quantum_info import Statevector
from qiskit.circuit.library import QuantumVolume
import mthree

In [2]:
from qiskit.test.mock import FakeAthens
noisy_sim = FakeAthens()

QV is defined in terms of heavy-ouputs of a distribution.  Heavy-outputs are those bit-strings that are those that have probabilities above the median value of the distribution.  Below we define the projection operator onto the set of bit-strings that are heavy-outputs for a given distribution.

In [3]:
def heavy_projector(qv_probs):
    """Forms the projection operator onto the heavy-outputs of a given probability distribution.
    
    Parameters:
        qv_probs (dict): A dictionary of bitstrings and associated probabilities.
        
    Returns:
        dict: Projector onto the heavy-set.
    """
    median_prob = np.median(list(qv_probs.values()))
    heavy_strs = {}
    for key, val in qv_probs.items():
        if val > median_prob:
            heavy_strs[key] = 1
    return heavy_strs

First we begin by calibrating our M3 mitigator over all the qubits on our noisy simulator. (We use the `independent` method since it is much faster to do so on a simulator.)

In [4]:
mit = mthree.M3Mitigation(noisy_sim)
mit.cals_from_system(method='independent')

Now we generate 10 QV circuits as our dataset.

In [5]:
# Generate QV circuits
N = 10
qv_circs = [QuantumVolume(5) for _ in range(N)]

Next, we have to determine the heavy-set of each circuit from the ideal answer, and then pass this along to our heavy-set projector function that we defined above.

In [6]:
# Compute ideal distributions and projectors on the heavy set.
ideal_probs = [Statevector.from_instruction(circ).probabilities_dict() for circ in qv_circs]
heavy_projectors = [heavy_projector(probs) for probs in ideal_probs]

Now, in preparation for actual execution, we add measurements to the end of our QV circuits, and compile them for the target device

In [7]:
# Add meauserements to circuits and transpile
trans_circs = transpile([circ.measure_all(inplace=False) for circ in qv_circs], noisy_sim,
                        layout_method='sabre', routing_method='sabre', optimization_level=3)

Because the SWAP mapping of the circuit permutes the qubit states, we need the final measurement mapping for each circuit to know which physical qubit corresponds to each measured bit:

In [8]:
# Determine final qubit mappings
qubits = [list(mthree.utils.final_measurement_mapping(circ)) for circ in trans_circs]

We are now ready to execute the circuits on the target backend, and mitigate the resulting raw counts:

In [9]:
# Execute circuits and mitigate
raw_counts = noisy_sim.run(trans_circs, shots=8192).result().get_counts()
quasi_collection = mit.apply_correction(raw_counts, qubits)

The value needed to determine if each circuit represents a passing QV value is determined by the expectation value of the heavy projector for each circuit:

In [10]:
# Determine expectation value of heavy set prjector
expvals = quasi_collection.expval(heavy_projectors)
expvals

array([0.64273103, 0.73990926, 0.7367156 , 0.68290608, 0.7421638 ,
       0.66162356, 0.70204107, 0.67630205, 0.73943566, 0.73140044])

A passing QV score is one where the expectation value is above 2/3:

In [12]:
# Check if the scores are passing or not
expvals > 2/3

array([False,  True,  True,  True,  True, False,  True,  True,  True,
        True])